In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
import pandas as pd
import io
import pickle
# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
print(tf.__version__)


2.15.0


In [ ]:
#name intake 'inp' and pfc [etoh] 'obs'
inp = pd.read_csv(('ncintakecomb.csv'), header=None) #intakes from MD data and published Englemann data
obs = pd.read_csv(('etohengcomb.csv'), header=None) #MD data from our and published Engleman data
inpEphys = pd.read_csv(('ephysitpncintake.csv'), header=None) #intakes for ensure + etoh
inpEnsure = pd.read_csv(('enIntakeForRNN.csv'), header=None) #intakes for ensure only

#reshape so rows are subjects/batches, as the RNN likes to read it
re_inp = np.expand_dims(np.transpose(inp.copy()), axis=-1)
re_obs = np.expand_dims(np.transpose(obs.copy()), axis=-1)
re_inpEphys = np.expand_dims(np.transpose(inpEphys.copy()), axis=-1)
re_inpEnsure = np.expand_dims(np.transpose(inpEnsure.copy()), axis=-1)

In [ ]:
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras import Sequential, layers

nodenum = [5, 10, 20, 30] #the different node numbers to generate/test
# Define base model function
def create_new_model_instance(nodes):
    model = Sequential()
    model.add(layers.SimpleRNN(nodes, input_shape=(24, 1), return_sequences=True)) #Use SimpleRNN notation for RNN layer
    model.add(layers.Dense(1)) #one Dense layer
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_absolute_error') #use MAE for the loss function
    return model

# Define the kfolds run and parameters
# kfolds where k=5 for each node num - 50 runs per node number
# note: this takes a LONG time to run, ~20 hours for my machine
def perform_xval(re_inp, re_obs, nodes):
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True)
    xval_results = []

    for train_index, val_index in kf.split(re_inp):
        inp_train, inp_val, obs_train, obs_val = train_test_split(
            re_inp[train_index], re_obs[train_index], test_size=0.2)

        # Create a new model instance for each fold
        # 5 folds total
        model_instance = create_new_model_instance(nodes)  
        model_instance.fit(inp_train, obs_train, epochs=2000, batch_size=3, verbose=0)
        
        obs_pred = model_instance.predict(inp_val).ravel()
        obs_val = obs_val.ravel()

        explained_var = explained_variance_score(obs_val, obs_pred)
        xval_results.append((explained_var, model_instance))

    return xval_results

num_runs = 50
kfoldsruns = np.empty((num_runs, len(nodenum), 5), dtype=object)  # Cache all 5 instances and scores per fold
best_model_instances = np.empty((num_runs, len(nodenum), 2), dtype=object)  # Cache best model instances per run by exp var in separate array

# iterate through runs
for run in range(num_runs):
    for idx, nodes in enumerate(nodenum):
        # Perform k-fold cross-validation and obtain results for all folds
        xval_results = perform_xval(re_inp, re_obs, nodes)

        # Extract and cache all fold results
        for fold_idx, (explained_var, model_instance) in enumerate(xval_results):
            kfoldsruns[run, idx, fold_idx] = (explained_var, model_instance)
        
        # Identify the best model instance from this set of folds
        best_fold = max(xval_results, key=lambda x: x[0])
        best_model_instances[run, idx, 0] = best_fold[0]  # Best explained variance
        best_model_instances[run, idx, 1] = best_fold[1]  # Best model instance

    # Progress update
    progress_percentage = (run + 1) / num_runs * 100
    print(f"Run {run + 1}/{num_runs} completed ({progress_percentage:.2f}%)")

In [ ]:
num_runs, num_nodes, _ = best_model_instances.shape

# Create an array to store the sorted indices for each run and node
sorted_indices = np.argsort(best_model_instances[:, :, 0], axis=0)[::-1]

# Create empty arrays to store sorted results
sorted_kfoldsruns = np.empty_like(best_model_instances)
sorted_model_instances_list = []

# Iterate through nodes
for node_idx in range(num_nodes):
    # Iterate through runs and use sorted indices to rearrange kfoldsruns
    for run in range(num_runs):
        sorted_run_idx = sorted_indices[run, node_idx]
        
        # Sort kfoldsruns based on explained variance
        sorted_kfoldsruns[run, node_idx] = best_model_instances[sorted_run_idx, node_idx]

        # Extract and store model instance directly from kfoldsruns
        sorted_model_instances_list.append(best_model_instances[sorted_run_idx, node_idx, 1])

# Display the sorted results
for node_idx in range(num_nodes):
    print(f"\nNode {nodenum[node_idx]} - Sorted Results:")
    for run in range(num_runs):
        avg_explained_var = sorted_kfoldsruns[run, node_idx, 0]
        model_instance = sorted_model_instances_list[node_idx * num_runs + run]
        print(f"Run {run + 1}: Explained Variance = {avg_explained_var:.4f}, Model Instance: {model_instance}")

top_10_models_data_node_20 = list(zip(sorted_kfoldsruns[:, 2, 0][:10], sorted_kfoldsruns[:, 2, 1][:10]))


In [ ]:
#Save the kfoldruns and model instances to a workspace to be used for intake only runs

kfoldsworkspace = {
    'kfoldsruns': kfoldsruns,
    'best20nodemodels': top_10_models_data_node_20,
    'best_model_instances': best_model_instances
}

# Save the workspace to a file name
with open('kfolds_workspace2.pkl', 'wb') as file:
    pickle.dump(kfoldsworkspace, file)

print("Successfully saved pkl workspace")